# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

### Setup workspace and experiment

In [2]:
ws = Workspace.from_config()

# print some information about the workspace
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'heart-failure-experiment'

experiment=Experiment(ws, experiment_name)

Workspace name: quick-starts-ws-161946
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-161946


### Create a cluster

Reuse the AutoML cluster or create a new one, if it doesn't exist

In [3]:
cluster_name = "expcluster"

# Use existing cluster, if it exists
try:
    compute_target = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',
                                                          max_nodes=6, min_nodes=1)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Create Dataset
Reuse the dataset from the AutoML run or upload it, if it doesn't exist.

In [4]:
from train import read_data
dataset=read_data()
df = dataset.to_pandas_dataframe()
df.describe()

AttributeError: '_OfflineRun' object has no attribute 'experiment'

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, randint

early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--n_estimators": randint(10000),
    "--max_depth": choice(10, 100, 1000, 5000),
    "--min_samples_split": choice(2,20,25,50)})

from azureml.core import Environment
from azureml.core import ScriptRunConfig

env = Environment.from_conda_specification(name = 'env', file_path = './envs/conda_environment.yml')

src = ScriptRunConfig(source_directory = "./",
    script = "train.py",
    compute_target = "expcluster",
    environment = env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=5)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.get_children_sorted_by_primary_metric()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details: ", best_hyperdrive_run.get_details())
print("best run metrics :", best_hyperdrive_run.get_metrics())

In [59]:
#TODO: Save the best model
for f in best_hyperdrive_run.get_file_names():
    if f.startswith('outputs/hyperdrive_model.onnx'):
        output_file_path = os.path.join('./model', 'hyperdrive_model.onnx')
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_hyperdrive_run.download_file(name=f, output_file_path=output_file_path)

In [56]:
best_hyperdrive_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_d962ebf8dd84ca36aa3417869ac656d6530c05b96c2d00749484d1ec5e169bf8_d.txt',
 'azureml-logs/65_job_prep-tvmps_d962ebf8dd84ca36aa3417869ac656d6530c05b96c2d00749484d1ec5e169bf8_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_d962ebf8dd84ca36aa3417869ac656d6530c05b96c2d00749484d1ec5e169bf8_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/97_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/hyperdrive_model.onnx']

### Convert to ONNX Format

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [29]:
# get environment yml and scoring scripts
for f in best_hyperdrive_run.get_file_names():
    if f.__contains__("scoring"):
        output_file_path = os.path.join('./model', os.path.basename(f))
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_hyperdrive_run.download_file(name=f, output_file_path=output_file_path)
    if f.__contains__("conda_env"):
        output_file_path = os.path.join('./model', os.path.basename(f))
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_automl_hyperdrive.download_file(name=f, output_file_path=output_file_path)

In [32]:
best_hyperdrive_run.properties

{'_azureml.ComputeTargetType': 'amlcompute',
 'ContentSnapshotId': 'ef2c10b8-6218-4527-a550-71a0c769c601',
 'ProcessInfoFile': 'azureml-logs/process_info.json',
 'ProcessStatusFile': 'azureml-logs/process_status.json'}

In [9]:
# this cell is how to register the .pkl model
description = "HyperDrive run heart-failure classification model"

from azureml.core import Model
model = Model.register(workspace=ws,
    model_name="hyperdrive_model",
    model_path='./model/hyperdrive_model.onnx',
    model_framework=Model.Framework.ONNX,
    model_framework_version='1.3',
    description=description)

Registering model hyperdrive_model


In [13]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model

service_name = 'heart-failure-hyperdrive-service'

inference_config = InferenceConfig(entry_script='model/scoring_file_v_1_0_0.py')
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                enable_app_insights=True,
                                                auth_enabled=True)

In [14]:
# deploy to local for debugging
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=6789)
test_service = Model.deploy(
    ws,
    name='test-service',
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)
test_service.wait_for_deployment(show_output=True)

AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# get some testdata to send a request
data = df.head(1).drop("HeartDisease", axis=1).to_json(orient='records')
body = {"data": data,
       "method": "predict"}
print(body)

In [ ]:
import json, requests
# test against local deploymenz
uri = test_service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
response = requests.post(uri, data=json.dumps(body), headers=headers)
print(response.json())

In [ ]:
# local deployment is working
test_service.delete()

In [ ]:
# send request to deployed web service
uri = service.scoring_uri
print(uri)
print(service.swagger_uri)
key, _ = service.get_keys()

In [ ]:
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"
response = requests.post(uri, data=json.dumps(body), headers=headers)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

